In [1]:
!pip install transformers datasets

In [2]:
% cd '/content/drive/MyDrive/Final Year Project/'
% ls

/content/drive/MyDrive/Final Year Project
 data/   models/                Resources.gdoc   vectorizer.joblib
 logs/  'Proposal Draft.gdoc'   scripts/         wrongly_classified_images.csv


In [3]:
# Load Data
import pandas as pd
pd.set_option('display.max_colwidth', None)

data = pd.read_csv('data/memotion_dataset_7k/labels.csv')
data.image_name = data.image_name.str.lower()
data = data.set_index('image_name', drop=False)
data

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
image_name,,,,,,,,,
image_1.jpg,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIKUT TREND PLAY THE 10 YEARS CHALLENGE AT FACEBOOK imgflip.com,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIKUT TREND PLAY THE 10 YEARS CHALLENGE AT FACEBOOK imgflip.com,hilarious,general,not_offensive,not_motivational,very_positive
image_2.jpeg,1,image_2.jpeg,The best of #10 YearChallenge! Completed in less the 4 years. Kudus to @narendramodi ji 8:05 PM - 16 Jan 2019 from Mumbai,The best of #10 YearChallenge! Completed in less the 4 years. Kudus to @narendramodi ji 8:05 PM - 16 Jan 2019 from Mumbai India,not_funny,general,not_offensive,motivational,very_positive
image_3.jpg,2,image_3.jpg,Sam Thorne @Strippin ( Follow Follow Saw everyone posting these 2009 vs 2019 pics so here's mine 6:23 PM - 12 Jan 2019 O 636 Retweets 3,Sam Thorne @Strippin ( Follow Follow Saw everyone posting these 2009 vs 2019 pics so here's mine 6:23 PM - 12 Jan 2019 O 636 Retweets 3 224 LIKES 65 636 3.2K,very_funny,not_sarcastic,not_offensive,not_motivational,positive
image_4.png,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
image_5.png,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious 10 Year Challenge Memes | What is #10 Year Challenge?,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious 10 Year Challenge Memes | What is #10 Year Challenge?,hilarious,very_twisted,very_offensive,not_motivational,neutral
...,...,...,...,...,...,...,...,...,...
image_6988.jpg,6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's Friday is the Chinese New Year To those who celebrate in full force; I'll be waiting for you at the end of the week with a digestive blend. ESSENTIALOILSTYLE.COM,Tuesday is Mardi Gras Wednesday is Valentine's Friday is the Chinese New Year To those who celebrate in full force; I'll be waiting for you at the end of the week with a digestive blend. ESSENTIALOILSTYLE.COM,very_funny,twisted_meaning,very_offensive,motivational,neutral
image_6989.jpg,6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MAANAGARAM A HANDFUL OF DIVERSE CHARACTERS FIND THEMSELVES IN EXTRAORDINARY SITUATIONS THAT ARE INTERLINKED WITH ONE ANOTHER,MUST WATCH MOVIES OF 2017 ITI Chennai memes MAANAGARAM A HANDFUL OF DIVERSE CHARACTERS FIND THEMSELVES IN EXTRAORDINARY SITUATIONS THAT ARE INTERLINKED WITH ONE ANOTHER,funny,twisted_meaning,not_offensive,not_motivational,neutral
image_6990.png,6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMPLAINING WORRYING DOUBTING LAZING AROUND FROWNING INSECURITY WEAKNESS IGNORANCE HATE UNGRATEFULNESS LISTENING DOING TEA SALADS ENCOURAGING HOPING BELIEVING WORKING OUT SMILING TRUST CONFIDENCE UNDERSTANDING LOVE GRATITUDE An easy guide to making the rest of 2017 the best of 2017...,LESS MORE TALKING PLANNING SODA JUNK FOOD COMPLAINING WORRYING DOUBTING LAZING AROUND FROWNING INSECURITY WEAKNESS IGNORANCE HATE UNGRATEFULNESS LISTENING DOING TEA SALADS ENCOURAGING HOPING BELIEVING WORKING OUT SMILING TRUST CONFIDENCE UNDERSTANDING LOVE GRATITUDE An easy guide to making the rest of 2017 the best of 2017...,funny,general,slight,not_motivational,positive


In [4]:
url_regex = '(https?:\/\/)?([\dA-Za-z\.-]+)\.([A-Za-z\.]{2,6})([\/\w \.-]*)'
data['text_cleaned'] = data['text_corrected'].str.replace(url_regex, '', regex=True)

In [5]:
anno = pd.read_csv('data/memotion_dataset_7k/annotations/annotations.csv')
anno.columns = ['index', 'image', 'labels']
del anno['index']
anno = anno.drop_duplicates().sort_values('image', ignore_index=True)
anno.labels = anno.labels.apply(eval)
anno = anno.groupby('image').sum()

In [6]:
all_labels = set()
for labels in anno.labels:
    labels = set() if type(labels) != set else labels
    all_labels.update(labels)
all_labels -= {None}

meme = set()
for label in all_labels:
    if 'meme' in label.lower():
        meme.add(label)

In [7]:
EXCLUDED_KEYWORDS = meme
EXCLUDED_KEYWORDS.update({'Image', 'Imgur', None, 'Humour', 'Text','Stock photography'})

In [8]:
data = data.join(anno)

In [9]:
train_texts, train_labels = data.text_cleaned.astype(str).to_list(), data.overall_sentiment.astype(str).to_list()
set(train_labels)
train_labels = [2 if 'positive' in x else 1 if 'negative' in x else 0 for x in train_labels]

In [10]:
import torch
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TextClassificationPipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [11]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [12]:
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [13]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [14]:
class MemotionCaptionDataset(torch.utils.data.Dataset):
    def __init__(self, texts, encodings, labels):
        self.texts = texts
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['text'] = self.texts[idx]
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MemotionCaptionDataset(train_texts, train_encodings, train_labels)
val_dataset = MemotionCaptionDataset(val_texts, val_encodings, val_labels)

In [15]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [16]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='models/roberta-finetuned',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,
    
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics
)
trainer.train()
trainer.evaluate()

***** Running training *****
  Num examples = 5593
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1750


Step,Training Loss
200,0.940100
400,0.900600
600,0.893600
800,0.894900
1000,0.893900
1200,0.893100
1400,0.896700
1600,0.893700


Saving model checkpoint to models/roberta-finetuned/checkpoint-500
Configuration saved in models/roberta-finetuned/checkpoint-500/config.json
Model weights saved in models/roberta-finetuned/checkpoint-500/pytorch_model.bin
Saving model checkpoint to models/roberta-finetuned/checkpoint-1000
Configuration saved in models/roberta-finetuned/checkpoint-1000/config.json
Model weights saved in models/roberta-finetuned/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to models/roberta-finetuned/checkpoint-1500
Configuration saved in models/roberta-finetuned/checkpoint-1500/config.json
Model weights saved in models/roberta-finetuned/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 1399
  Batch size = 64


{'epoch': 10.0,
 'eval_accuracy': 0.5889921372408864,
 'eval_loss': 0.8972378969192505,
 'eval_runtime': 4.6306,
 'eval_samples_per_second': 302.123,
 'eval_steps_per_second': 4.751}

In [17]:
pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=False, device=0)
pipeline

outputs = [out['label'] for out in pipeline(val_texts)]
pd.Series(outputs).value_counts()

Disabling tokenizer parallelism, we're using DataLoader multithreading already
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


LABEL_2    1399
dtype: int64

In [18]:
id2label = {'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2}

In [19]:
from sklearn.metrics import f1_score
y_true = val_labels
y_pred = [id2label[out] for out in outputs]
f1_score(y_true, y_pred, average='macro')

0.24711351027140502

In [20]:
def check_accuracy(model_name, y_true, y_pred):
    print("Accuracy of " + model_name + ": {}%".format(round(accuracy_score(y_true, y_pred)*100,2)))
    print("\nConfusion Matrix of " + model_name + ":\n")
    print(confusion_matrix(y_true, y_pred))
    print("\nClassification Report of " + model_name + ":\n")
    print(classification_report(y_true, y_pred))
    print("F1 Macro Score: " + str(f1_score(y_true, y_pred, average='macro')))

#row True col predicted